# Visualizing traffic flow through Azure Firewall

Here we just want to pull data from Log Analytics attached to Azure Firewall and visualize traffic flows it sees

but first - lets import some JS :)

In [ ]:
d3 = await import("https://cdn.jsdelivr.net/npm/d3@7/+esm");
d3co = await import("https://cdn.jsdelivr.net/npm/d3-color@3.1.0/+esm");
d3a = await import("https://cdn.jsdelivr.net/npm/d3-array@3.2.4/+esm");
d3c = await import("https://cdn.jsdelivr.net/npm/d3-collection@1.0.7/+esm");
d3p = await import("https://cdn.jsdelivr.net/npm/d3-path@3.1.0/+esm")
d3sh = await import("https://cdn.jsdelivr.net/npm/d3-shape@3.2.0/+esm");
d3sa = await import("https://cdn.jsdelivr.net/npm/d3-sankey@0/+esm");
d3f = await import("https://cdn.jsdelivr.net/npm/d3-fetch@3.0.1/+esm");

now it is time to pull our data from Azure

In [ ]:
# this sets some parameters for the PS script
WorkspaceID = '2cc447f2-d22b-4f8c-8786-f522710c1b25'
$dateFilter = 'ago(1d)'

In [ ]:
# this executes Log Analytics query and pulls data
$query = @"
AZFWApplicationRule | where TimeGenerated >= $dateFilter
"@

$data = Invoke-AzOperationalInsightsQuery -WorkspaceId $WorkspaceID -Query $query -ErrorAction Stop | Select-Object -ExpandProperty Results

$sourceIpGroups = $data | Group-Object SourceIp

$dataSet = 
foreach ($group in $sourceIpGroups) {
    $source = $group.Name
    $targets = $group.Group | Group-Object DestinationIp, Action

    foreach($target in $targets) {
        [PSCustomObject]@{
            source = $source;
            sourceName = $source
            target = $target.Group[0].DestinationIp
            action = $target.Group[0].Action
            value = $target.Count
        }
    }
}

Now, we should transform data, so it can be consumed by d3js.

In [ ]:
# some quick transformations to prepare data for d3ks-sankey + exporting to json
$i = 0
$nodes = ($data.source + $data.target) | Select-Object -Unique

$links = $data | % {
    [pscustomobject]@{
        source = $nodes.IndexOf($_.source)
        target = $nodes.IndexOf($_.target)
        value = $_.value
    }
}

[pscustomobject]@{
    nodes = $nodes | % { [pscustomobject]@{name = $_} }
    links = $links
} | ConvertTo-Json -Depth 10 | out-file "traffic-data/data.json"

Prepare a canvas

In [ ]:
<div style="display: flex; justify-content: space-between;">
    <div id="graph-container" style="flex-grow: 1;"></div>
</div>

and visualize!

This imports the `json` file we just exported, and shows it in the canvas above

In [ ]:
// visualize!

const width = 800;
const height = 2000;

var data = await d3f.json("traffic-data/data.json");

// Create the SVG container.
const svg = d3.select('#graph-container')
    .append('svg')
    .attr("width", width)
    .attr("height", height)
    .call(d3.zoom().on("zoom", (event) => {
        svg.attr("transform", event.transform);
    }))
    .append("g");

drawSankey();

function drawSankey() {
    var sankey = d3sa.sankey()
        .nodeAlign(d3sa.sankeyLeft)
        .nodeWidth(20)
        .nodePadding(20)
        .extent([[1, 50], [width - 1, height - 5]]);
    
    var graph = sankey(data)

    const color = d3.scaleOrdinal(d3.schemeSet3);

    // Drawing nodes
    const rect = svg.append("g")
        .attr("stroke", "#000")
        .selectAll("rect")
        .data(graph.nodes)
        .join("rect")
        .attr("x", d => d.x0)
        .attr("y", d => d.y0)
        .attr("height", d => d.y1 - d.y0 >= 3 ? d.y1 - d.y0 : 3)
        .attr("width", d => d.x1 - d.x0)
        .attr("fill", d => color(d.name));
    
    rect.append("title")
        .text(d => `${d.name}\n${d.targetLinks.length > 0 ? d.targetLinks.map(o => o.source.name).join("\n") : ""}`);

    // Creating gradients for links
    const defs = svg.append("defs");
    graph.links.forEach((link, i) => {
        const gradient = defs.append("linearGradient")
            .attr("id", "gradient" + i)
            .attr("gradientUnits", "userSpaceOnUse")
            .attr("x1", link.source.x1)
            .attr("x2", link.target.x0);

        gradient.append("stop")
            .attr("offset", "0%")
            .attr("stop-color", color(link.source.name));

        gradient.append("stop")
            .attr("offset", "100%")
            .attr("stop-color", color(link.target.name));
    });

    // Drawing links with gradient
    svg.append("g")
        .attr("fill", "none")
        .attr("stroke-opacity", 0.5)
        .selectAll("path")
        .data(graph.links)
        .join("path")
        .attr("d", d3sa.sankeyLinkHorizontal())
        .attr("stroke", (d, i) => `url(#gradient${i})`)
        .attr("stroke-width", d => Math.max(1, d.width))
        .append("title")
        .text(d => `${d.source.name} → ${d.target.name}`);

    // Drawing labels for the nodes
    svg.append("g")
        .selectAll("text")
        .data(graph.nodes)
        .join("text")
        .attr("x", d => d.x0 < width / 2 ? d.x1 + 6 : d.x0 - 6)
        .attr("y", d => (d.y1 + d.y0) / 2)
        .attr("dy", "0.35em")
        .attr("text-anchor", d => d.x0 < width / 2 ? "start" : "end")
        .attr("font-size", "15px")
        .text(d => d.name);
}